# This contains the methods and models for the paper "Has machine learning over-promised in healthcare?"

It contains the models: 
- ozkan (Model A in the paper): PCA with nearest neighbours https://doi.org/10.3390/e18040115
- caliskan (Model B in the paper): stacked auto encoder https://electricajournal.org/Content/files/sayilar/58/3311-3318.pdf
- ulhaq (Model C in the paper): SVM with feature selection https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8672565

This is to look at the each inflatory effect that was discussed in the manuscript, breaking down how much inflation to the performance it causes

## Load and prepare/preprocess data
OPD is the Oxford Parkinsons Disease dataset, retrieved from UCI. This was originally created by Max Little, and is used extensively in PD classification research (https://archive.ics.uci.edu/ml/datasets/parkinsons)

mPower is the larger dataset, from the mPower study, a Parkinsons mobile application developed by Sage Bionetworks and described in Synpase (doi:10.7303/syn4993293).

In [ ]:
import pandas as pd
import dataHandler as dh
from os.path import join

OPD_samples = pd.read_csv(join('Data','OPD_data.csv'))
OPD_participants = pd.read_csv(join('Data','OPD_participants.csv'))

print('One of the participants (S31) does not have demographic data (i.e., age and gender), which is why their submissions are missing from counts that include anything to do with gender or age\n')

dh.OPD_summary(OPD_samples, OPD_participants)

#Include the S31 patient here because we dont use age or sex, so it is fine to include this person for this experiment
OPD_participants.at[31,['participant','status']] = ('S31',1)

In [ ]:
import dataLoader as dl
import mPower_data_adjustments as mda

#load the original submission data, the features extracted from them, and information needed to filter them
submissions = dl.pickleLoad(join('Data','submissions_Full.pickle'))
features = dl.pickleLoad(join('Data','combinedFeatures.pickle'))
d2 = dl.pickleLoad(join('Data','d2Feature5.pickle'))
rms_energy = dl.pickleLoad(join('Data','rms_energy_notNormalised.pickle'))

features.rename(columns={'recordID':'recordId'},inplace=True)
d2 = d2[d2['d2'] != 'nan'] #drop nans in d2

#filter out bad submissions and see how many remains
rms_energy = rms_energy[rms_energy['rms_1_mean'] > 300] #250
rms_energy = rms_energy[rms_energy['rms_1_std'] < 2000] #2500
rms_energy = rms_energy[rms_energy['energy_1_mean'] > 50000] #45000
features = features[features['Degree of voice breaks (%)'] < 30] #40
features = features[features['Fraction of locally unvoiced frames (%)'] < 30]#40

#combine them and see how many remains
mPower_samples = pd.merge(rms_energy['recordId'],submissions,on='recordId')
mPower_samples = pd.merge(mPower_samples,features,on='recordId')
mPower_samples = mPower_samples.merge(d2[['recordId','d2']],on='recordId',how='inner')
mPower_samples = mPower_samples[~(mPower_samples['gender'] == 'Prefer not to answer')] #retains only male and female

#Remove samples that I found were bad, by looking at the 10 most extreme values for all 22 features and listening
mPower_samples = mda.remove_bad_samples(mPower_samples)

#Recalculate spread1 and spread2, the original was wrong.
mPower_samples = mda.recalculate_spreads(mPower_samples)

#Adjust the scale of several mPower features to be on the same scale as OPD. 
#Things like Jitter (%) is currently 2%, but in OPD features would be 0.02
mPower_samples = mda.adjust_feature_scale(mPower_samples)

mPower_samples.index = range(len(mPower_samples))
mPower_participants = mPower_samples.drop_duplicates(subset=['healthCode'])[['healthCode', 'age', 'diagnosis-year','gender','onset-year','professional-diagnosis']]

#print summary of the dataset
dh.mPower_summary(mPower_samples, mPower_participants)

In [ ]:
from ozkan_model import ozkan
from caliskan_model import caliskan
from ulhaq_model import ulhaq

import itertools
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import KFold, train_test_split
import torch.nn as nn
from datetime import date
from os.path import join

import evaluation
import resultsHandler as rh

## Using the OPD data

In [ ]:
kfold_splits = 8
training_split=0.75 #its meant to be 0.7, but 0.75 results in clean proportions with participants (there is no integer value for 70% of 8 participants)
repetitions = 30
#repetitions = 3

###################### Seeds ######################
seeds = list(range(repetitions))
###################################################


######################## to_numpy method ########################

def to_numpy(OPD_samples):
    
    features = ['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP', 
                'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 
                'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']
    
    X = OPD_samples[features].to_numpy()
    y = OPD_samples['status'].to_numpy()
    
    return X, y

#################################################################


#The model we will be using for Ozkan is "ozkan PCA_14 k_1 MinMaxScaler X only"
#This was the best model for 10fold CV, and best for 70/30 split

#The model we will be using for Caliskan is "caliskan ReLU Sigmoid latent:6, epochs:400, lr:0.0030 MinMaxScaler X only"
#This was the 2nd best model for 10fold CV, and best for 70/30 split

#The model we will be using for Ul-Haq is "ulhaq rbf gamma:0.2000 C:5 num_features:14 StandardScaler X only"
#This was the 2nd best model for both 10fold CV and 70/30 split

######################## Global settings ########################
preprocessors = [MinMaxScaler,StandardScaler]
preprocessing_methods = ['X only']
global_settings = list(itertools.product(preprocessors,preprocessing_methods))
##################################################################

######################## Ozkan settings ########################
components = [14]
ks = [1]
ozkan_settings = list(itertools.product(components,ks))
################################################################

######################## Caliskan settings ########################
lrses = [[0.003]*4]
epochses = [[400]*4]
rhoses = [[0.15,0.25],]
lamses = [[0.03,0.03],]
Bses = [[2,2],]
activationses = [[nn.ReLU,nn.Sigmoid]]
latent_sizes = [6,]
caliskan_settings = list(itertools.product(lrses,epochses,rhoses,lamses,Bses,activationses,latent_sizes))
###################################################################

######################## Ul-Haq settings ########################
kernels = ['rbf']
gammas = [0.2]
Cs = [5]
num_featureses = [14] #best at 10 in paper
ulhaq_settings = list(itertools.product(kernels,gammas,Cs,num_featureses))
#################################################################

### First the unmodified results (no inflatory effects removed)

In [ ]:
kf_results_OPD_0 = dl.pickleLoad(join('Results','repeated_kfold with OPD unmodified 25October21.pickle'))
split_results_OPD_0 = dl.pickleLoad(join('Results','repeated_traintest with OPD unmodified25October21.pickle'))

rh.print_results(kf_results_OPD_0,top=3)
rh.print_results(split_results_OPD_0,top=3)


### Participant independence (removing the digital fingerprinting effect)

In [ ]:
samples=OPD_samples
participants=OPD_participants

ozkan_method=ozkan
caliskan_method=caliskan
ulhaq_method=ulhaq

seeds=seeds
n_splits=kfold_splits
repetitions=repetitions
verbose_odds=0
dataset = 'OPD'

def kf_method(X, y, samples, participants):
    
    kf = list(KFold(n_splits=n_splits, shuffle=True).split(participants))
    indices = []
    for participant_train_index,participant_test_index in kf:
        train_participants = participants.iloc[participant_train_index]['participant']
        test_participants = participants.iloc[participant_test_index]['participant']
        
        cond1 = samples['participant'].isin(train_participants)
        cond2 = samples['participant'].isin(test_participants)
        
        train_indices = samples.loc[cond1].index.tolist()
        test_indices = samples.loc[cond2].index.tolist()
        
        indices.append((train_indices,test_indices))
    return indices

test_size = 1.0 - training_split
def split_method(X, y, samples, participants):
    split = train_test_split(participants,test_size=test_size)
    
    (train_participants,test_participants) = [c['participant'] for c in split]
    cond1 = samples['participant'].isin(train_participants)
    cond2 = samples['participant'].isin(test_participants)
    
    train_indices = samples.loc[cond1].index.tolist()
    test_indices = samples.loc[cond2].index.tolist()
    
    return X[train_indices],X[test_indices],y[train_indices],y[test_indices]



kf_results_OPD_1,split_results_OPD_1 = evaluation.evaluate(dataset, kf_method, split_method, samples, participants, to_numpy, global_settings, ozkan_settings, caliskan_settings, ulhaq_settings, ozkan_method, caliskan_method, ulhaq_method, repetitions, verbose_odds, seeds)

display(kf_results_OPD_1)
display(split_results_OPD_1)

d = date.today().strftime("%d%B%y")
dl.pickleSave(kf_results_OPD_1,join('Results','kfold OPD digital fingerprinting removed ' + d + '.pickle'))
dl.pickleSave(split_results_OPD_1,join('Results','traintest OPD digital fingerprinting removed' + d + '.pickle'))

### Participant independence and class balance (removing the digital fingerprinting and accuracy paradox effect)

In [ ]:
samples=OPD_samples
participants=OPD_participants

ozkan_method=ozkan
caliskan_method=caliskan
ulhaq_method=ulhaq

seeds=seeds
n_splits=kfold_splits
repetitions=repetitions
verbose_odds=0
dataset = 'OPD'

def restrict_PD_samples(PD_samples,PD_participants):
    restricted_PD_samples = pd.DataFrame()
    #I know I need to remove 2/3 of all PD samples to match the sizes
    for p in PD_participants['participant']: restricted_PD_samples = restricted_PD_samples.append(dh.df_retrieve(PD_samples,{'participant':p}).sample(n=2))
    return restricted_PD_samples

def kf_method(X, y, samples, participants):
    control_samples = dh.df_retrieve(samples,{'status':0})
    PD_samples = dh.df_retrieve(samples,{'status':1})
    control_participants = dh.df_retrieve(participants,{'status':0})
    PD_participants = dh.df_retrieve(participants,{'status':1})

    restricted_PD_samples = restrict_PD_samples(PD_samples,PD_participants)
    samples = restricted_PD_samples.append(control_samples)

    kf_control = list(KFold(n_splits=n_splits, shuffle=True).split(control_participants))
    kf_PD = list(KFold(n_splits=n_splits, shuffle=True).split(PD_participants))
    
    indices = []
    for (control_train_index,control_test_index),(PD_train_index,PD_test_index) in zip(kf_control,kf_PD):
        control_train_participants = control_participants.iloc[control_train_index]['participant']
        control_test_participants = control_participants.iloc[control_test_index]['participant']
        PD_train_participants = PD_participants.iloc[PD_train_index]['participant']
        PD_test_participants = PD_participants.iloc[PD_test_index]['participant']
        
        cond1 = samples['participant'].isin(control_train_participants) | samples['participant'].isin(PD_train_participants)
        cond2 = samples['participant'].isin(control_test_participants) | samples['participant'].isin(PD_test_participants)
        
        train_indices = samples.loc[cond1].index.tolist()
        test_indices = samples.loc[cond2].index.tolist()
        
        indices.append((train_indices,test_indices))

    return indices

test_size = 1.0 - training_split
def split_method(X, y, samples, participants):
    control_samples = dh.df_retrieve(samples,{'status':0})
    PD_samples = dh.df_retrieve(samples,{'status':1})
    control_participants = dh.df_retrieve(participants,{'status':0})
    PD_participants = dh.df_retrieve(participants,{'status':1})

    restricted_PD_samples = restrict_PD_samples(PD_samples,PD_participants)
    samples = restricted_PD_samples.append(control_samples)
    
    split_control = train_test_split(control_participants,test_size=test_size)
    split_PD = train_test_split(PD_participants,test_size=test_size)
    
    (control_train_participants,control_test_participants) = [c['participant'] for c in split_control]
    (PD_train_participants,PD_test_participants) = [c['participant'] for c in split_PD]

    cond1 = samples['participant'].isin(control_train_participants) | samples['participant'].isin(PD_train_participants)
    cond2 = samples['participant'].isin(control_test_participants) | samples['participant'].isin(PD_test_participants)

    train_indices = samples.loc[cond1].index.tolist()
    test_indices = samples.loc[cond2].index.tolist()
    
    return X[train_indices],X[test_indices],y[train_indices],y[test_indices]
    #X_train, X_test, y_train, y_test

kf_results_OPD_2,split_results_OPD_2 = evaluation.evaluate(dataset, kf_method, split_method, samples, participants, to_numpy, global_settings, ozkan_settings, caliskan_settings, ulhaq_settings, ozkan_method, caliskan_method, ulhaq_method, repetitions, verbose_odds, seeds)

display(kf_results_OPD_2)
display(split_results_OPD_2)

d = date.today().strftime("%d%B%y")
dl.pickleSave(kf_results_OPD_2,join('Results','kfold OPD digital fingerprinting and accuracy paradox removed ' + d + '.pickle'))
dl.pickleSave(split_results_OPD_2,join('Results','traintest OPD digital fingerprinting and accuracy paradox removed' + d + '.pickle'))

### All effects removed

In [ ]:
kf_results_OPD_3 = dl.pickleLoad(join('Results','repeated_kfold with OPD modified 25October21.pickle'))
split_results_OPD_3 = dl.pickleLoad(join('Results','repeated_traintest with OPD modified25October21.pickle'))

rh.print_results(kf_results_OPD_3,top=1)
rh.print_results(split_results_OPD_3,top=1)

### Using the mPower data

In [ ]:
kfold_splits = 5
training_split = 0.7
repetitions = 10

###################### Seeds ######################
seeds = list(range(repetitions))
###################################################


######################## to_numpy method ########################

def to_numpy(mPower_samples):

    features = ['Mean pitch (Hz)','Minimum pitch (Hz)','Maximum pitch (Hz)','Jitter (local) (%)','Jitter (local, absolute)','Jitter (rap) (%)','Jitter (ppq5) (%)','Jitter (ddp) (%)',
                'Shimmer (local) (%)','Shimmer (local, dB) (dB)','Shimmer (apq3) (%)','Shimmer (apq5) (%)','Shimmer (apq11) (%)','Shimmer (dda) (%)',
                'Mean noise-to-harmonics ratio','Mean harmonics-to-noise ratio (dB)','spread1 (negative entropy of F0)','spread2 (standard error of F0)','PPE','DFA','RPDE','d2']
    
    X = mPower_samples[features].to_numpy()
    y = (mPower_samples['professional-diagnosis']*1).to_numpy(dtype='int64')
    
    return X, y

#################################################################


#The model we will be using for Ozkan is "ozkan PCA_16 k_11 StandardScaler X only"
#This was the best model for both 10fold CV and 70/30 split

#The model we will be using for Caliskan is "caliskan ReLU Sigmoid latent:6, epochs:50, lr:0.0030 StandardScaler X only"
#This was the best model for both 10fold CV and 70/30 split

#The model we will be using for Ulhaq is "ulhaq rbf gamma:scale C:10 num_features:20 StandardScaler X only"
#This was the 2nd best model for 10fold CV and best for 70/30 split


######################## Global settings ########################
preprocessors = [StandardScaler]
preprocessing_methods = ['X only']
global_settings = list(itertools.product(preprocessors,preprocessing_methods))
##################################################################

######################## Ozkan settings ########################
components = [16]
ks = [11]
ozkan_settings = list(itertools.product(components,ks))
################################################################

######################## Caliskan settings ########################
lrses = [[0.003]*4,]
epochses = [[50]*4,]
rhoses = [[0.15,0.25],]
lamses = [[0.03,0.03],]
Bses = [[2,2],]
activationses = [[nn.ReLU,nn.Sigmoid],]
latent_sizes = [6,]
caliskan_settings = list(itertools.product(lrses,epochses,rhoses,lamses,Bses,activationses,latent_sizes))
###################################################################

######################## Ul-Haq settings ########################
kernels = ['rbf']
gammas = ['scale']
Cs = [10]
num_featureses = [20] #best at 10 in paper
ulhaq_settings = list(itertools.product(kernels,gammas,Cs,num_featureses))
#################################################################

### First the unmodified results (no inflatory effects removed)

In [ ]:
kf_results_mPower_0 = dl.pickleLoad(join('Results','repeated_kfold with mPower unmodified 24January22.pickle'))
split_results_mPower_0 = dl.pickleLoad(join('Results','repeated_traintest with mPower unmodified24January22.pickle'))

rh.print_results(kf_results_mPower_0,top=2)
rh.print_results(split_results_mPower_0,top=2)

### Participant independence (removing the digital fingerprinting effect)

In [ ]:
samples=mPower_samples
participants=mPower_participants

ozkan_method=ozkan
caliskan_method=caliskan
ulhaq_method=ulhaq

seeds=seeds
n_splits=kfold_splits
repetitions=repetitions
verbose_odds=0
dataset = 'mPower'

def kf_method(X, y, samples, participants):
    kf = list(KFold(n_splits=n_splits, shuffle=True).split(participants))
    indices = []
    for participant_train_index,participant_test_index in kf:
        train_participants = participants.iloc[participant_train_index]['healthCode']
        test_participants = participants.iloc[participant_test_index]['healthCode']
        
        cond1 = samples['healthCode'].isin(train_participants)
        cond2 = samples['healthCode'].isin(test_participants)
        
        train_indices = samples.loc[cond1].index.tolist()
        test_indices = samples.loc[cond2].index.tolist()
        
        indices.append((train_indices,test_indices))
    return indices

test_size = 1.0 - training_split
def split_method(X, y, samples, participants):
    split = train_test_split(participants,test_size=test_size)
    
    (train_participants,test_participants) = [c['healthCode'] for c in split]
    cond1 = samples['healthCode'].isin(train_participants)
    cond2 = samples['healthCode'].isin(test_participants)
    
    train_indices = samples.loc[cond1].index.tolist()
    test_indices = samples.loc[cond2].index.tolist()
    
    return X[train_indices],X[test_indices],y[train_indices],y[test_indices]


kf_results_mPower_1,split_results_mPower_1 = evaluation.evaluate(dataset, kf_method, split_method, samples, participants, to_numpy, global_settings, ozkan_settings, caliskan_settings, ulhaq_settings, ozkan_method, caliskan_method, ulhaq_method, repetitions, verbose_odds, seeds)

display(kf_results_mPower_1)
display(split_results_mPower_1)

d = date.today().strftime("%d%B%y")
dl.pickleSave(kf_results_mPower_1,join('Results','kfold mPower digital fingerprinting removed ' + d + '.pickle'))
dl.pickleSave(split_results_mPower_1,join('Results','traintest mPower digital fingerprinting removed' + d + '.pickle'))

### Participant independence and class balance (removing the digital fingerprinting and accuracy paradox effect)

In [ ]:
import mPower_modifications as mm
import numpy as np
samples=mPower_samples
participants=mPower_participants

ozkan_method=ozkan
caliskan_method=caliskan
ulhaq_method=ulhaq

seeds=seeds
n_splits=kfold_splits
repetitions=repetitions
verbose_odds=0
dataset = 'mPower'

def kf_method(X, y, samples, participants):
    
    PD_samples = dh.df_retrieve(samples,{'professional-diagnosis':True})
    control_samples = dh.df_retrieve(samples,{'professional-diagnosis':False})
    
    target = min(len(PD_samples),len(control_samples))
    PD_samples = PD_samples.sample(n=target)
    control_samples = control_samples.sample(n=target)
    
    PD_folds = mm.create_k_folds(PD_samples,k=n_splits)
    control_folds = mm.create_k_folds(control_samples,k=n_splits)

    PD_mapping = {}
    control_mapping = {}
    for fold in range(n_splits):
        PD_mapping.update(dict.fromkeys(PD_folds.loc[fold,'participants'],fold))
        control_mapping.update(dict.fromkeys(control_folds.loc[fold,'participants'],fold))
    
    PD_samples['fold'] = PD_samples['healthCode'].map(PD_mapping)
    control_samples['fold'] = control_samples['healthCode'].map(control_mapping)
    mapping = {key:val for key,val in list(zip(PD_samples['recordId'],PD_samples['fold']))}
    mapping.update({key:val for key,val in list(zip(control_samples['recordId'],control_samples['fold']))})
        
    samples['fold'] = samples['recordId'].map(mapping)
        
    fold_indices = [np.where(samples['fold']==fold)[0] for fold in range(n_splits)]

    folds = []
    
    iter_i = np.random.permutation(len(fold_indices))
    for i in iter_i:
        test_indices = fold_indices[i]
        train_indices = [fold_indices[j] for j in iter_i[iter_i != i]]
        train_indices = list(itertools.chain.from_iterable(train_indices))
        folds.append((train_indices,test_indices))
    

    return folds

test_size = 1.0 - training_split
def split_method(X, y, samples, participants):
    
    PD_samples = dh.df_retrieve(samples,{'professional-diagnosis':True})
    control_samples = dh.df_retrieve(samples,{'professional-diagnosis':False})
    
    target = min(len(PD_samples),len(control_samples))
    PD_samples = PD_samples.sample(n=target)
    control_samples = control_samples.sample(n=target)
    
    PD_folds = mm.create_k_folds(PD_samples,k=10)
    control_folds = mm.create_k_folds(control_samples,k=10)
    

    PD_mapping = {}
    control_mapping = {}
    for fold in range(10):
        PD_mapping.update(dict.fromkeys(PD_folds.loc[fold,'participants'],fold))
        control_mapping.update(dict.fromkeys(control_folds.loc[fold,'participants'],fold))
    
    PD_samples['fold'] = PD_samples['healthCode'].map(PD_mapping)
    control_samples['fold'] = control_samples['healthCode'].map(control_mapping)
    mapping = {key:val for key,val in list(zip(PD_samples['recordId'],PD_samples['fold']))}
    mapping.update({key:val for key,val in list(zip(control_samples['recordId'],control_samples['fold']))})
        
    samples['fold'] = samples['recordId'].map(mapping)
        
    fold_indices = [np.where(samples['fold']==fold)[0] for fold in range(10)]
    
    #combine 3 random folds
    test_indices = []
    for _ in range(3):
        i = np.random.randint(0,len(fold_indices))
        test_indices.append(fold_indices[i])
        fold_indices.pop(i)
        
    train_indices = [idx for sublist in fold_indices for idx in sublist]
    test_indices = [idx for sublist in test_indices for idx in sublist]
    
    return X[train_indices],X[test_indices],y[train_indices],y[test_indices]



kf_results_mPower_2,split_results_mPower_2 = evaluation.evaluate(dataset, kf_method, split_method, samples, participants, to_numpy, global_settings, ozkan_settings, caliskan_settings, ulhaq_settings, ozkan_method, caliskan_method, ulhaq_method, repetitions, verbose_odds, seeds)

display(kf_results_mPower_2)
display(split_results_mPower_2)

d = date.today().strftime("%d%B%y")
dl.pickleSave(kf_results_mPower_2,join('Results','kfold mPower digital fingerprinting and accuracy paradox removed ' + d + '.pickle'))
dl.pickleSave(split_results_mPower_2,join('Results','traintest mPower digital fingerprinting and accuracy paradox removed' + d + '.pickle'))

### With all effects removed

In [ ]:
kf_results_mPower_3 = dl.pickleLoad(join('Results','repeated_kfold with mPower modified 25January22.pickle'))
split_results_mPower_3 = dl.pickleLoad(join('Results','repeated_traintest with mPower modified25January22.pickle'))

rh.print_results(kf_results_mPower_3,top=1)
rh.print_results(split_results_mPower_3,top=1)
